In [1]:
import pathlib
import pandas as pd
import numpy as np
import swifter
from tqdm import tqdm

from itertools import *

### Goal: Process Industry Data Data --> .tsv for choropleth

Current US choropleth example takes .tsv with the following format:

```
id	rate
01001	5.1
01003	4.9
01005	8.6
01007	6.2
```

where id is the county FIPS and rate is some value for coloring the choropleth

In [2]:
# Helper Functions
def fips_to_str(fips):
    fips_str = str(int(fips))

    while len(fips_str) < 5:
        fips_str = '0' + fips_str

    return fips_str


def extract_state_fips(fips_str):
    return int(fips_str[0:2])


def contains_3digit_naics(row):
    '''return True if any of the relevant naics is a three digit naic'''
    naics_list = row['relevant_naics'].replace(' ', '').strip('[]').split(',')
    check = any([len(naic)==3 for naic in naics_list])
    return check

In [3]:
base_dir = pathlib.Path.cwd()

# Load list of all county fips
county_fips_file = base_dir.parent.parent / 'data_raw' / 'counties_fips.csv'
counties = pd.read_csv(county_fips_file)
counties['fips_str'] = counties.FIPS.apply(fips_to_str)

# Load the cancer incidence data
data_dir = base_dir.parent.parent / 'data_clean' 

industry_data_csv = data_dir / 'indicators_per-industry_per-county.csv'

# Load ALL incidence rate data to dataframe
df = pd.read_csv(industry_data_csv)
df['fips_str'] = df['fips'].apply(fips_to_str)

# df3 = df[df.apply(contains_3digit_naics, axis=1)]

# df3.head()

# df = df.loc[0:10000, :]
# df.shape

In [4]:
df.head()

,Unnamed: 0,fips,county,name,industry_code,industry_detail,relevant_naics,year,payann,estab,...,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR,fips_str
0,0,8001,1,"Adams County, Colorado",113000,timber and raw forest products,[113],2012,0,1,...,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,08001
1,1,8001,1,"Adams County, Colorado",115000,agriculture and forestry support,[115],2012,290,6,...,0.0,3.405586,0.0,0.000183,0.008769,0.95877,2.011564,211.423463,15.653502,08001
2,2,8001,1,"Adams County, Colorado",211000,unrefined oil and gas,[211],2012,0,6,...,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,08001
3,3,8001,1,"Adams County, Colorado",2122a0,"iron, gold, silver, and other metal ores","[21221, 21222, 21229]",2012,0,1,...,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,08001
4,4,8001,1,"Adams County, Colorado",212310,dimensional stone,[21231],2012,0,4,...,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,08001


In [5]:
df.columns

Index(['Unnamed: 0', 'fips', 'county', 'name', 'industry_code',
       'industry_detail', 'relevant_naics', 'year', 'payann', 'estab', 'emp',
       'ACID', 'ENRG', 'ETOX', 'EUTR', 'FOOD', 'GCC', 'HAPS', 'HAZW', 'HC',
       'HNC', 'HRSP', 'HTOX', 'JOBS', 'LAND', 'METL', 'MINE', 'MSW', 'NREN',
       'OZON', 'PEST', 'REN', 'SMOG', 'VADD', 'WATR', 'fips_str'],
      dtype='object')

In [6]:

# Extract the 2-digit naics classification for each entry
df['relevant_naics'] = df['industry_code'].apply(lambda x: int(x[0:2]))

# extract and aggregate the relevant data
id_cols =        ['fips', 'relevant_naics']
bigness_cols =   ['emp', 'payann', 'estab']
indicator_cols = ['ACID', 'ENRG', 'ETOX', 'EUTR', 'FOOD', 'GCC', 'HAPS', 'HAZW', 'HC',
                  'HNC', 'HRSP', 'HTOX', 'JOBS', 'LAND', 'METL', 'MINE', 'MSW', 'NREN',
                  'OZON', 'PEST', 'REN', 'SMOG', 'VADD', 'WATR']

df2 = (df[id_cols + bigness_cols + indicator_cols]
       .groupby(by=['fips','relevant_naics'], as_index=False)
       .sum()
       .rename(columns={'fips':'id'})
      )

df2.head()

,id,relevant_naics,emp,payann,estab,ACID,ENRG,ETOX,EUTR,FOOD,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1001,11,30,1102,6,0.861581,0.275217,893.717406,0.277371,0.818711,...,0.000066,0.0,3.210888,0.000000,3.767853e-07,0.012181,0.275217,20.041190,654.873075,136.234634
1,1001,21,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2,1001,22,182,16657,9,284.165607,728617.651665,204.647213,23.516440,6.054180,...,0.047113,0.0,23.518978,435169.166702,1.886537e-03,0.000006,293448.484963,2400.143851,11995.153334,7473.552102
3,1001,31,0,0,5,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
4,1001,32,0,0,7,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
#
# Fill Missing Rows
#

def fill_missing_rows(df):

    n_rows = len(df.id.unique()) * len(df.relevant_naics.unique())
    product_iter = product(df.id.unique(), df.relevant_naics.unique())

    for id_val, naics_val in tqdm(product_iter, total=n_rows):
        check_exists = len(df.query(f'id=={id_val} & relevant_naics=={naics_val}'))

        if check_exists < 1:
            new_entry = pd.DataFrame(data={'id':id_val, 'relevant_naics': naics_val}, index=[0])
            yield new_entry

df_missing = pd.concat(fill_missing_rows(df2)).reset_index(drop=True)
df2 = (df2
       .append(df_missing)
       .fillna(0)
       .reset_index(drop=True)
      )

100%|██████████| 65940/65940 [04:03<00:00, 270.91it/s]


In [7]:
df2

,id,relevant_naics,emp,payann,estab,ACID,ENRG,ETOX,EUTR,FOOD,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1001,11,30,1102,6,0.861581,0.275217,893.717406,0.277371,0.818711,...,0.000066,0.0,3.210888,0.000000,3.767853e-07,1.218061e-02,0.275217,20.041190,654.873075,136.234634
1,1001,21,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
2,1001,22,182,16657,9,284.165607,728617.651665,204.647213,23.516440,6.054180,...,0.047113,0.0,23.518978,435169.166702,1.886537e-03,5.996926e-06,293448.484963,2400.143851,11995.153334,7473.552102
3,1001,31,0,0,5,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,1001,32,0,0,7,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60013,56045,61,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
60014,56045,62,101,2145,24,0.003288,0.043918,0.005317,0.000638,7.678576,...,0.000041,0.0,53.119870,0.000000,1.295951e-08,7.996942e-09,0.043918,0.130185,1450.711675,0.655140
60015,56045,71,5,135,3,0.067368,0.107925,34.657328,0.040056,1.816929,...,0.000011,0.0,5.620021,0.000000,1.975391e-06,9.302177e-04,0.107925,2.744965,71.922583,6.531894
60016,56045,72,28,1438,23,0.006143,0.043024,0.260775,0.004386,27.025234,...,0.000012,0.0,57.259023,0.000000,1.579423e-06,1.823160e-09,0.043024,0.686682,756.297604,2.082064


In [8]:
naics_codes = pd.read_csv(base_dir.parent.parent / 'data_raw' / '2-digit_2012_NAICS_Codes.csv')
naics_codes.rename(columns={'2012 NAICS US   Code': 'code', 
                            '2012 NAICS US Title': 'details'},
                   inplace=True)

# naics_codes[naics_codes.code.apply(lambda x: True if len(x.split('-'))>1 else False)]
# naics_codes.set_index('2012 NAICS US   Code', inplace=True)
# naics_codes.columns

df2.relevant_naics = df2.relevant_naics.apply(lambda x: str(x).split('-')[0])
df2.head()

,id,relevant_naics,emp,payann,estab,ACID,ENRG,ETOX,EUTR,FOOD,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1001,11,30,1102,6,0.861581,0.275217,893.717406,0.277371,0.818711,...,0.000066,0.0,3.210888,0.000000,3.767853e-07,0.012181,0.275217,20.041190,654.873075,136.234634
1,1001,21,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2,1001,22,182,16657,9,284.165607,728617.651665,204.647213,23.516440,6.054180,...,0.047113,0.0,23.518978,435169.166702,1.886537e-03,0.000006,293448.484963,2400.143851,11995.153334,7473.552102
3,1001,31,0,0,5,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
4,1001,32,0,0,7,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
def flatten_multi_naics(naic_code):
    naic_code = int(naic_code)
    if naic_code in (31, 32, 33):
        return 31
    elif naic_code in (44, 45):
        return 44
    elif naic_code in (48, 49):
        return 48
    else:
        return naic_code



df2.relevant_naics = df2.relevant_naics.swifter.apply(flatten_multi_naics)

df2.relevant_naics.unique()

/Users/merrelbook/anaconda3/envs/datalab/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Pandas Apply: 100%|██████████| 60018/60018 [00:00<00:00, 460313.96it/s]


array([11, 21, 22, 31, 44, 48, 51, 52, 53, 54, 55, 56, 61, 62, 71, 72, 81])

In [10]:
def expand_range(x):
    if '-' in x:
        n = x.split('-')
        return str([l for l in range(int(n[0]), int(n[1])+1)]).strip('[]')
    else:
        return x


    
naics_2 = naics_codes[naics_codes.code.apply(lambda x: True if (len(x)==2) or ('-' in x) else False)]
naics_2.code = naics_2.code.apply(expand_range)

def lookup_naics_details(row):
    n = row.relevant_naics
#     print(str(int(n)))
    try:
        d = naics_2[naics_2.code.str.contains(str(int(n)))]['details'].values[0]
    except IndexError:
        print(n)
        d = n
    
    return d


/Users/merrelbook/anaconda3/envs/datalab/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
naics_2

,code,details
0,11,"Agriculture, Forestry, Fishing and Hunting"
131,21,"Mining, Quarrying, and Oil and Gas Extraction"
179,22,Utilities
204,23,Construction
277,"31, 32, 33",Manufacturing
929,42,Wholesale Trade
1094,"44, 45",Retail Trade
1261,"48, 49",Transportation and Warehousing
1401,51,Information
1479,52,Finance and Insurance


In [12]:
lookup_naics_details(df2.loc[3,:])

'Manufacturing'

In [13]:
df2['industry_detail'] = df2.swifter.apply(lookup_naics_details, axis=1)

df2['id'] = df2['id'].swifter.apply(fips_to_str)

df2.head()

Dask Apply: 100%|██████████| 16/16 [00:09<00:00,  1.62it/s]
/Users/merrelbook/anaconda3/envs/datalab/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Pandas Apply: 100%|██████████| 60018/60018 [00:00<00:00, 358581.79it/s]


,id,relevant_naics,emp,payann,estab,ACID,ENRG,ETOX,EUTR,FOOD,...,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR,industry_detail
0,01001,11,30,1102,6,0.861581,0.275217,893.717406,0.277371,0.818711,...,0.0,3.210888,0.000000,3.767853e-07,0.012181,0.275217,20.041190,654.873075,136.234634,"Agriculture, Forestry, Fishing and Hunting"
1,01001,21,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,"Mining, Quarrying, and Oil and Gas Extraction"
2,01001,22,182,16657,9,284.165607,728617.651665,204.647213,23.516440,6.054180,...,0.0,23.518978,435169.166702,1.886537e-03,0.000006,293448.484963,2400.143851,11995.153334,7473.552102,Utilities
3,01001,31,0,0,5,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,Manufacturing
4,01001,31,0,0,7,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,Manufacturing


In [14]:
df2.to_csv(
    data_dir / 'tables_for_viz' / 'final_industry_byCounty_byType.tsv',
    index=False, sep='\t'
)

In [16]:
df_key = (df2[['relevant_naics', 'industry_detail']]
          .groupby('industry_detail').first()
          .sort_values(by='relevant_naics'))

df_key.to_csv(
    data_dir / 'tables_for_viz' / 'industry_ID_list.csv',
#     index=False
)

df_key.head()

,relevant_naics
industry_detail,
"Agriculture, Forestry, Fishing and Hunting",11
"Mining, Quarrying, and Oil and Gas Extraction",21
Utilities,22
Manufacturing,31
Retail Trade,44


In [17]:
df2

,id,relevant_naics,emp,payann,estab,industry_detail
0,01001,11,30.0,1102.0,6.0,"Agriculture, Forestry, Fishing and Hunting"
1,01001,21,0.0,0.0,2.0,"Mining, Quarrying, and Oil and Gas Extraction"
2,01001,22,182.0,16657.0,9.0,Utilities
3,01001,31,0.0,0.0,5.0,Manufacturing
4,01001,31,0.0,0.0,7.0,Manufacturing
...,...,...,...,...,...,...
65935,56043,55,0.0,0.0,0.0,Management of Companies and Enterprises
65936,56043,48,0.0,0.0,0.0,Transportation and Warehousing
65937,56045,31,0.0,0.0,0.0,Manufacturing
65938,56045,31,0.0,0.0,0.0,Manufacturing


In [ ]:
len(df.industry_code.unique())